<a href="https://colab.research.google.com/github/ahodelin/Woche2AbelHodelinHernandez/blob/main/Abel_Hodelin_Hernandez_ETLAufgaben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# import libraries
import pandas as pd
import sqlite3
from sqlite3 import Error

In [46]:
# mount drive to access data
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
# material_path = "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material"
material_path = "/content/drive/MyDrive/csv_files/allergy/"

In [48]:
!ls {material_path} # !ls "/content/drive/Shareddrives/05_BIDS_Datenmanagement & Archivierung im Umfeld der Forschung/Material/source_dbs"

 allergy_test.db	 encounters.gsheet	    patients.csv
'careplans (1).gsheet'	 imaging_studies.csv	    payers.csv
 careplans.csv		 imaging_studies.gsheet     payer_transitions.csv
 careplans.gsheet	 immunizations.csv	    procedures.csv
 conditions.csv		 medications.csv	    procedures.gsheet
 devices.csv		'observations (1).gsheet'   providers.csv
 disease.csv		 observations.csv	    source_allergy_test.db
 disease.gsheet		 observations.gsheet	    supplies.csv
 encounters.csv		 organizations.csv


# Python SQLite Guide


In [49]:
# SQLite connection
def create_connection_memory():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
        conn = sqlite3.connect(':memory:')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close()

In [50]:
# create a temporary empty database in the memory.
conn = create_connection_memory()
cur = conn.cursor()
print("Successfully Connected to SQLite")


Successfully Connected to SQLite


In [51]:
# Create SQLite table
create_table_query = '''CREATE TABLE SqliteDb_developers (
                                id INTEGER PRIMARY KEY,
                                name TEXT NOT NULL,
                                email text NOT NULL UNIQUE,
                                joining_date datetime,
                                salary REAL NOT NULL);'''

cur.execute(create_table_query)
conn.commit()
print("SQLite table created")

SQLite table created


In [52]:
# list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('SqliteDb_developers',)]


In [53]:
# insert into table
cur.execute("INSERT INTO SqliteDb_developers (id,  name, email, salary) VALUES (1, 'name1', 'name1@gmail.com', 1000)")


In [54]:
# insert into table-Larger example
rows = [(2, 'name2', 'name2@gmail.com', '2006-02-28', 2000),
        (3, 'name3', 'name3@gmail.com', '2006-03-05', 3000),
        (4, 'name4', 'name4@gmail.com', '2006-04-06', 4000)]
cur.executemany('insert into SqliteDb_developers values (?,?,?,?,?)', rows)
conn.commit()


In [55]:
select_query = "SELECT * FROM SqliteDb_developers"
cur.execute(select_query)
records = cur.fetchall()
print("Total rows in table: ", len(records))
for row in records:
  print(row)

Total rows in table:  4
(1, 'name1', 'name1@gmail.com', None, 1000.0)
(2, 'name2', 'name2@gmail.com', '2006-02-28', 2000.0)
(3, 'name3', 'name3@gmail.com', '2006-03-05', 3000.0)
(4, 'name4', 'name4@gmail.com', '2006-04-06', 4000.0)


In [56]:
# select with parameters
select_query = """select * from SqliteDb_developers where id = ?"""
cur.execute(select_query, (1,))
records = cur.fetchall()
print("Printing ID ", id)
for row in records:
    print("Name = ", row[1])
    print("Email  = ", row[2])
    print("JoiningDate  = ", row[3])
    print("Salary  = ", row[4])

Printing ID  <built-in function id>
Name =  name1
Email  =  name1@gmail.com
JoiningDate  =  None
Salary  =  1000.0


In [57]:
# update table
cur.execute("UPDATE SqliteDb_developers SET Email=? WHERE id =?", ('Uname2@gmail.com', 2))
print("Number of rows updated: {}".format(cur.rowcount))

Number of rows updated: 1


In [58]:
# access to metadata (tables and columns) using PRAGMA
cur.execute("PRAGMA table_info(SqliteDb_developers);")
data = cur.fetchall()
# print(data)
for d in data:
  print(d[0], d[1], d[2])

0 id INTEGER
1 name TEXT
2 email text
3 joining_date datetime
4 salary REAL


In [59]:
# Excute whole SQL code in one step
cur.executescript("""
        DROP TABLE IF EXISTS patient_table;
        CREATE TABLE patient_table(patient_id INT, name TEXT, diagnose Text, birthdate datetime);
        INSERT INTO patient_table VALUES(10, 'name1', 'Asthma', '2006-02-28');
        INSERT INTO patient_table VALUES(20, 'name1', 'Asthma', '2007-02-28');
        INSERT INTO patient_table VALUES(30, 'name1', 'Asthma', '2008-02-28');
        INSERT INTO patient_table VALUES(40, 'name1', 'Asthma', '2009-02-28');
        DELETE FROM patient_table WHERE patient_id > 30;
     """)
conn.commit()
# cur.execute("DELETE FROM SqliteDb_developers WHERE id < 3")
cur.execute("SELECT COUNT(patient_id) FROM patient_table")
cur.fetchone()


(3,)

There are more information in:

http://zetcode.com/db/sqlitepythontutorial/

https://pynative.com/python-sqlite/

https://stackoverflow.com/questions/228912/sqlite-parameter-substitution-problem

# Exercises

### Connect to the database and see how many tables it contains




In [60]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

In [61]:
# src_db = f"{material_path}/source_dbs/source_allergy_test.db"
src_db = f"{material_path}/source_allergy_test.db"

src_conn = create_connection(src_db)
src_cur = src_conn.cursor()
print("Database connection is successfully")

Database connection is successfully


### Query the tables, join two tables



In [62]:
# Join Table of patiens, observations and conditions wiht pseudonyms for Patient Id
patient_observation_condition = pd.read_sql_query("select ps.ID PATIENT_ID, p.GENDER, p.RACE, o.CODE OBSERVATIONCODE, o.DESCRIPTION OBSERVATIONDESCRIPTION, c.CODE CONDITIONCODE, c.DESCRIPTION CONDITIONDESCRIPTION  FROM patients p join observations o on p.Id = o.PATIENT \
JOIN (select id PATIENT, 'P'||random() ID from patients) as ps on ps.PATIENT = p.Id JOIN conditions c on c.PATIENT = o.PATIENT order by o.CODE;" , src_conn
  ) 
patient_observation_condition.head(4)

,PATIENT_ID,GENDER,RACE,OBSERVATIONCODE,OBSERVATIONDESCRIPTION,CONDITIONCODE,CONDITIONDESCRIPTION
0,P3753049111862834341,M,white,10230-1,Left ventricular Ejection fraction,88805009,Chronic congestive heart failure (disorder)
1,P5403099086071242295,M,white,10230-1,Left ventricular Ejection fraction,162864005,Body mass index 30+ - obesity (finding)
2,P9083070284773106907,M,white,10230-1,Left ventricular Ejection fraction,403191005,Second degree burn
3,P-3097872635000778074,M,white,10230-1,Left ventricular Ejection fraction,444814009,Viral sinusitis (disorder)


### Copy a table from one database to another(create empty SQLite database)

In [63]:
src_new_db = f"/content/drive/MyDrive/etl_db/new_allergy_test.db"
#!ls "/content/drive/MyDrive/etl_db/"
src_new_conn = create_connection(src_new_db)
src_new_cur = src_new_conn.cursor()
print("Database connection is successfully")


Database connection is successfully


In [64]:
patient_observation_condition.to_sql('patients_observation_condition', con=src_new_conn, schema=None, if_exists='append', index=False)
patient_observation_condition_new_db=pd.read_sql_query("select * from patients_observation_condition;", src_new_conn)
patient_observation_condition_new_db.head(4)

,PATIENT_ID,GENDER,RACE,OBSERVATIONCODE,OBSERVATIONDESCRIPTION,CONDITIONCODE,CONDITIONDESCRIPTION
0,P3523390032818135460,M,white,10230-1,Left ventricular Ejection fraction,88805009,Chronic congestive heart failure (disorder)
1,P6371618240270824614,M,white,10230-1,Left ventricular Ejection fraction,162864005,Body mass index 30+ - obesity (finding)
2,P5605516810912704492,M,white,10230-1,Left ventricular Ejection fraction,403191005,Second degree burn
3,P-1033456409398049619,M,white,10230-1,Left ventricular Ejection fraction,444814009,Viral sinusitis (disorder)


### Transform ETL Job: Joint two table and write them into a new table

In [65]:
# see the above code cell


### Write ETL jobs for all Tables




### Write a Program that schedules the ETL jobs and execute them in order
